In [ ]:
!pip install flask


In [ ]:
from flask import Flask, request, redirect, render_template_string
import string
import random
import sqlite3

app = Flask(__name__)

# Connect to SQLite database
conn = sqlite3.connect(':memory:', check_same_thread=False)
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS url_mapping (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        short_url TEXT NOT NULL UNIQUE,
        long_url TEXT NOT NULL
    )
''')
conn.commit()


def generate_short_url():
    characters = string.ascii_letters + string.digits
    short_url = ''.join(random.choice(characters) for _ in range(6))
    return short_url


@app.route('/')
def index():
    return render_template_string('''
        <form method="POST" action="/shorten">
            <label for="long_url">Long URL:</label>
            <input type="text" name="long_url" id="long_url" required>
            <button type="submit">Shorten URL</button>
        </form>
    ''')


@app.route('/shorten', methods=['POST'])
def shorten_url():
    long_url = request.form.get('long_url')

    # Check if the long URL already exists in the database
    cursor.execute('SELECT short_url FROM url_mapping WHERE long_url = ?', (long_url,))
    existing_short_url = cursor.fetchone()

    if existing_short_url:
        return f'Shortened URL: {request.url_root}{existing_short_url[0]}'

    # Generate a unique short URL
    short_url = generate_short_url()

    # Store the mapping in the database
    cursor.execute('INSERT INTO url_mapping (short_url, long_url) VALUES (?, ?)', (short_url, long_url))
    conn.commit()

    return f'Shortened URL: {request.url_root}{short_url}'


@app.route('/<short_url>')
def redirect_to_long_url(short_url):
    # Retrieve the long URL from the database
    cursor.execute('SELECT long_url FROM url_mapping WHERE short_url = ?', (short_url,))
    long_url = cursor.fetchone()

    if long_url:
        return redirect(long_url[0])

    return 'URL not found'


# Run the Flask app
if __name__ == '__main__':
    app.run(port=5000)
